In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:
#|default_exp cli

In [ ]:
#|export
# Python native modules
import os
# Third party libs
from fastcore.all import *
# Local modules

# CLI
> Convenience CLI utils for fastrl projects

In [ ]:
#|export
@call_parse
def fastrl_make_requirements(
    path:Path=None, # The path to a dir with the settings.ini, if none, cwd.
    project_file:str='settings.ini', # The file to load for reading the requirements
    out_path:Path=None, # The output path (can be relative to `path`)
    verbose:bool=False # Output to stdout
):
    requirement_types = ['','dev_','pip_']
    path = ifnone(path, Path.cwd())/project_file

    if not path.exists(): raise OSError(f'File {path} does not exist')

    out_path = ifnone(out_path, Path('extra'))
    out_path = out_path if out_path.is_absolute() else path.parent/out_path
    out_path.mkdir(parents=True, exist_ok=True)
    if verbose: print('Outputting to path: ',out_path)
    config = Config(path.parent,path.name)

    for req in requirement_types:
        requirements = config[req+'requirements']
        requirements = requirements.replace(' ','\n')
        Path(out_path/(req+'requirements.txt')).write_text(requirements)

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()